# Problema 3
$$
\begin{array}{ll}
\operatorname{minimizar} & \frac{1}{2} \vec{\boldsymbol{x}}^{\top} G \vec{\boldsymbol{x}}+\vec{\boldsymbol{c}}^{\top} \vec{\boldsymbol{x}} \\
\text { sujeto a } & A \vec{x}=\vec{\boldsymbol{b}} \\
& x_{i} \geq \ell_{i} \quad \text { si } \ell_{i} \text { es finito, } \\
& x_{i} \leq u_{i} \quad \text { si } u_{i} \text { es finito. }
\end{array}
$$

- Definimos $J \subset I$ como sigue:

- Para $x_{j} \geq \ell_{j}$ definimos $\left|g_{j}\left(x_{0}\right)\right| \leq 8 \varepsilon_{m} \max \left\{\left|\ell_{j}\right|, 1\right\} \Longrightarrow j \in J$

- Para $x_{j} \leq u_{j}$ definimos $\left|g_{j}\left(x_{0}\right)\right| \leq 8 \varepsilon_{m} \max \left\{\left|u_{j}\right|, 1\right\} \Longrightarrow j \in J$

Donde $\varepsilon_{m}$ es el épsilon de la máquina `eps(Float64)`

In [1]:
include("Solvers.jl")
using .Solvers, LinearAlgebra, MAT, .Solvers.Utils

In [2]:
problem = matread("../data/lpi_woodinfe.mat")["Problem"]

Dict{String, Any} with 11 entries:
  "A"      => …
  "aux"    => Dict{String, Any}("c"=>[10.0; 11.0; … ; 15.0; 15.0], "hi"=>[100.0…
  "b"      => [0.0; 0.0; … ; 0.0; 0.0]
  "date"   => "1989"
  "name"   => "LPnetlib/lpi_woodinfe"
  "kind"   => "linear programming problem"
  "author" => "H. Greenberg"
  "id"     => 731.0
  "notes"  => ["An infeasible Netlib LP problem, in lp/infeas.  For more inform…
  "title"  => "Netlib LP problem woodinfe: minimize c'*x, where Ax=b, lo<=x<=hi"
  "ed"     => "J. Chinneck"

In [3]:
b = problem["b"] 
c = problem["aux"]["c"][:] # El [:] es para interpretar como vector
l = problem["aux"]["lo"][:] # El [:] es para interpretar como vector
u = problem["aux"]["hi"][:] # El [:] es para interpretar como vector
n_eq = length(b)
A_eq = problem["A"]
G = I(length(c))

# Como all(isfinite.(l)) == true, se complen todas las cotas inferiores
A = [A_eq; -I(length(l))]
b = [b; -l]

# Imponiendo cotas superiores
# Concatenando las filas I_j de una identidad tal que u_j < Inf
mask = isfinite.(u)
A = [A; I(length(u))[mask, :]]
# Igual cambiando b para que dimensiones coincidan
b = [b; u[mask]]
b = b[:]

138-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   ⋮
 200.0
 100.0
  25.0
  10.0
  50.0
  40.0
  20.0
   5.0
  30.0

In [4]:
# Definiendo el conjunto inicial de igualdades
# Obtenemos x_0 con linprog (pero ya sabemos que va a ser vec(0))
x_0 = linprog(A, b, n_eq)

# Shorthand para epsilon de maquina
εₘ = eps(Float64)

# Obteniendo los masks que indican si se usa la restricción
# Recordamos que para las cotas inferiores g_i(x) = l_i - x_i & para las sup g_i(x) = x_i - u_i
J_l = abs.(l - x_0) .<= 8 * εₘ *  max.(abs.(l), ones(length(l)))

# Ahora para las superiores, recordando quitar los renglones donde u_j no es finito (por size de A).
J_u = abs.(x_0 - u) .<= 8 * εₘ *  max.(abs.(u), ones(length(u)))
J_u = J_u[isfinite.(u)]

J = [J_l; J_u]

# Concatenando con un índice aleatorio
R_index = rand(findall(J), 1)
notJ = falses(size(A, 1) - n_eq)
notJ[R_index] .= true
W_0 = [trues(n_eq); notJ]

138-element BitVector:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [5]:
println(W_0)

Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Nótese que el número de filas de W_0 coincide con el de A, y el tamaño de b

In [6]:
@assert size(A, 1) == length(W_0)
@assert size(A, 1) == length(b)

In [7]:
iters, x_star, q_star, μ = activeSetMethod(G, c, A, b, n_eq, W_0)

α = -0.0, j = 38 

W_k = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 




[100.0, 100.0, 0.0, 0.0, 0.0, 150.0, 0.0, 0.0, 25.0, 10.0, 35.0, 50.0, 40.0, 20.0, 5.0, 15.0, 30.0, 65.0, 100.0, 100.0, 90.0, 50.0, 10.0, 20.0, 25.0, 10.0, 15.0, 0.0, 50.0, 40.0, 20.0, 5.0, 15.0, 20.0, 25.0, 30.0, 20.0, 0.0, 100.0, 35.0, 90.0, 0.0, 50.0, 0.0, 10.0, 15.0, 100.0, 15.0, 20.0, 0.0, 10.0, 20.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 10.0, 0.0, 25.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 15.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 30.0, 0.0, 15.0, 0.0, 20.0]

Rama 1. ||d_k|| = 158.44024128412227, q(x) = 155185.0, α=-0.0 k = 0
α = -0.0, j = 39 

W_k = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


Rama 1. ||d_k|| = 162.8318856073767, q(x) = 155185.0, α=-0.0 k = 10
α = -0.0, j = 92 

W_k = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 


[100.0, 100.0, 0.0, 0.0, 0.0, 150.0, 0.0, 0.0, 25.0, 10.0, 35.0, 50.0, 40.0, 20.0, 5.0, 15.0, 30.0, 65.0, 100.0, 100.0, 90.0, 50.0, 10.0, 20.0, 25.0, 10.0, 15.0, 0.0, 50.0, 40.0, 20.0, 5.0, 15.0, 20.0, 25.0, 30.0, 20.0, 0.0, 100.0, 35.0, 90.0, 0.0, 50.0, 0.0, 10.0, 15.0, 100.0, 15.0, 20.0, 0.0, 10.0, 20.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 10.0, 0.0, 25.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 15.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 30.0, 0.0,

q(x) = 155185.0, α=-0.0 k = 18
α = -0.0, j = 109 

W_k = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 


[100.0, 100.0, 0.0, 0.0, 0.0, 150.0, 0.0, 0.0, 25.0, 10.0, 35.0, 50.0, 40.0, 20.0, 5.0, 15.0, 30.0, 65.0, 100.0, 100.0, 90.0, 50.0, 10.0, 20.0, 25.0, 10.0, 15.0, 0.0, 50.0, 40.0, 20.0, 5.0, 15.0, 20.0, 25.0, 30.0, 20.0, 0.0, 100.0, 35.0, 90.0, 0.0, 50.0, 0.0, 10.0, 15.0, 100.0, 15.0, 20.0, 0.0, 10.0, 20.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 10.0, 0.0, 25.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 15.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 30.0, 0.0, 15.0, 0.0, 20.0]

Rama 1. ||d_k|| = 


[100.0, 100.0, 0.0, 0.0, 0.0, 150.0, 0.0, 0.0, 25.0, 10.0, 35.0, 50.0, 40.0, 20.0, 5.0, 15.0, 30.0, 65.0, 100.0, 100.0, 90.0, 50.0, 10.0, 20.0, 25.0, 10.0, 15.0, 0.0, 50.0, 40.0, 20.0, 5.0, 15.0, 20.0, 25.0, 30.0, 20.0, 0.0, 100.0, 35.0, 90.0, 0.0, 50.0, 0.0, 10.0, 15.0, 100.0, 15.0, 20.0, 0.0, 10.0, 20.0, 0.0, 0.0, 0.0, 0.0, 0.0, 15.0, 5.0, 0.0, 10.0, 0.0, 25.0, 0.0, 0.0, 5.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 10.0, 15.0, 0.0, 15.0, 5.0, 0.0, 0.0, 0.0, 0.0, 30.0, 0.0, 15.0, 0.0, 20.0]

Rama 1. ||d_k|| = 105.19117647058823, q(x) = 155185.0, α=0.0 k = 29
α = 0.0, j = 58 

W_k = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

...


α = 2.3529411764705674, j = 96 

W_k = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1] 


[51.40476190476162, 46.69047619047613, 49.90476190476167, 49.99999999999992, 9.999999999999943, 49.40476190476162, 42.69047619047613, 49.90476190476167, 25.000000000000025, 10.000000000000169, 34.9999999999993, 42.63636363636443, 40.00000000000037, 19.99999999999966, 4.999999999999846, 22.363636363637113, 29.999999999999886, 64.9999999999991, 100.00000000000023, 100.00000000000027, 90.00000000000018, 50.0000000000002, 9.99999999999983, 19.999999999999815, 25.000000000000085, 10.0, 14.9999999999999, 0.0, 50.000000000000114, 40.000000


α = 2.0765027322403986, j = 117 

W_k = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1] 


[51.40476190476155, 47.03446115288213, 49.560776942355574, 49.999999999999936, 9.999999999999915, 49.40476190476155, 43.03446115288213, 49.560776942355574, 25.000000000000053, 10.000000000000183, 34.999999999999126, 42.636363636364585, 40.000000000000384, 19.999999999999673, 4.999999999999818, 22.363636363637283, 29.999999999999858, 64.9999999999989, 100.00000000000024, 100.00000000000028, 90.0000000000002, 50.0000000000002, 9.999999999999801, 19.999999999999787, 25.000000000000142, 9.999999999999943, 14.999999999999872, 0.0, 50.00


Rama 1. ||d_k|| = 1.1901709401710754, q(x) = 108312.56959414853, α=4.20107719928158 
Rama 2. 
α = 3.165199729178903, j = 101 

W_k = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1] 


[51.404761904761415, 47.03446115288201, 49.560776942355474, 49.99999999999991, 9.999999999999858, 49.404761904761415, 43.03446115288201, 49.560776942355474, 25.00000000000008, 10.000000000000176, 34.99999999999894, 44.26363636363738, 34.46363636363738, 19.999999999999574, 4.999999999999754, 26.272727272728062, 29.99999999999983, 64.99999999999869, 100.00000000000031, 100.00000000000034, 90.00000000000004, 50.0000000000002, 9.999999999999773

j = 1, μ=0.0 


OptimizeResult{Float64}(77, [51.473684210525846, 46.99999999999977, 49.526315789473244, 49.999999999999936, 9.999999999999872, 49.473684210525846, 42.99999999999977, 49.526315789473244, 25.00000000000008, 10.00000000000019  …  10.706060606061115, 15.3333333333331, 4.701754385964762, 6.3636363636360045, 6.060606060606133, 30.00000000000044, 5.797971671190648e-13, 6.454545454545579, 9.811266330431206e-15, 9.33333333333296], 108291.05263157893, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  70.83333333333286, 0.0, 0.0, 17.72727272727112, 48.63636363636224, 0.0, 0.0, 22.500000000002142, 42.87878787878921, 53.66666666666505])

In [8]:
print(x_star)

[51.473684210525846, 46.99999999999977, 49.526315789473244, 49.999999999999936, 9.999999999999872, 49.473684210525846, 42.99999999999977, 49.526315789473244, 25.00000000000008, 10.00000000000019, 34.9999999999989, 44.263636363637424, 34.463636363637406, 19.999999999999538, 4.999999999999747, 26.272727272728105, 29.99999999999983, 64.99999999999868, 100.00000000000033, 100.00000000000038, 90.00000000000006, 50.00000000000021, 9.999999999999744, 19.9999999999997, 25.00000000000017, 9.99999999999983, 14.999999999999815, 0.0, 50.00000000000027, 40.000000000000156, 20.000000000000284, 5.000000000000185, 14.999999999999943, 20.000000000000085, 25.000000000000057, 30.00000000000037, 20.000000000000455, 0.0, 42.6929824561408, 20.715151515151803, 33.061403508771605, 6.069696969696821, 15.192982456140143, 2.4787878787882955, 9.999999999999348, 15.000000000000439, 43.798245614035025, 15.63636363636354, 25.31515151515177, 33.53508771929768, 3.636363636363539, 7.869696969696832, 12.666666666666217,

In [9]:
q_star

108291.05263157893